In [1]:
import numpy as np
import pandas as pd

## 가상 메뉴 데이터 (자스민 참고)
- 부산대 앞 먹어본 음료
- 73개의 음료로 구성

In [2]:
menu_df = pd.read_excel("fake_data.xlsx", sheet_name=0)

In [3]:
menu_df.sample(5)

,메뉴명,가격
39,오렌지 생과일요플레,3000
65,아이스티복숭아,2500
4,바닐라라떼,3000
36,딸기 생과일요플레,3000
0,에스프레소,1000


In [4]:
menu_df.shape

(73, 2)

In [5]:
menu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 2 columns):
메뉴명    73 non-null object
가격     73 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.3+ KB


## 가상 고객 데이터 (학원 명단 참조)
- 빅데이터 2기 수강생 +
- 교수님, 강사님, 멘토님
- 31명
- 이름, 성별(0: 남, 1: 여), 연령대(20대, 30대, 40대)

In [6]:
cus_df = pd.read_excel("fake_data.xlsx", sheet_name=1)

In [7]:
cus_df.sample(5)

,이름,성별,연령대
7,권재경,1,30
5,박소현,1,20
0,김효원,1,20
11,이환희,1,20
29,송길태,0,40


In [8]:
cus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 3 columns):
이름     31 non-null object
성별     31 non-null int64
연령대    31 non-null int64
dtypes: int64(2), object(1)
memory usage: 872.0+ bytes


## 가상 매출 데이터 생성

In [9]:
menuName = menu_df.메뉴명

In [10]:
name = cus_df.이름

In [11]:
# 랜덤 날짜 생성 함수
import random
from datetime import datetime, timedelta

# or a function
def gen_datetime(min_year=1900, max_year=datetime.now().year):
    # generate a datetime in format yyyy-mm-dd hh:mm:ss.000000
    start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=365 * years)
    return start + (end - start) * random.random()

In [12]:
# 랜덤 사이즈 만큼 생성
random_size = 100000 # 랜덤 매출 생성 수

# 난수 생성
menu_randint = np.random.randint(low=0, high=len(menuName), size=random_size)
name_randint = np.random.randint(low=0, high=len(name), size=random_size)
date_random = [ gen_datetime(2019, 2020) for i in range(random_size) ]

# 난수 대입
menus = [ menu_df.메뉴명[i] for i in menu_randint ]
names = [ cus_df.이름[i] for i in name_randint ]

# 가상 데이터 생성
tmp_df = pd.DataFrame( {"이름": names,
               "메뉴": menus,
               "판매시간": date_random
              })

# 판매시간 별 정렬
sales_df = tmp_df.sort_values(by='판매시간').reset_index().drop(columns=['index'])

In [13]:
sales_df

,이름,메뉴,판매시간
0,박성훈,오렌지 요거트,2019-01-01 00:18:51.007199
1,서준호,레몬차,2019-01-01 00:30:30.191679
2,김효원,토마토 생과일주스,2019-01-01 00:40:32.821089
3,김형건,자몽차,2019-01-01 00:50:41.264769
4,유주영,오렌지 생과일요플레,2019-01-01 00:55:59.278321
...,...,...,...
99995,류아인,블루레몬 에이드,2020-12-30 23:20:39.161649
99996,유주영,초코라떼,2020-12-30 23:26:31.944160
99997,최병철,얼그레이,2020-12-30 23:35:27.087424
99998,박경민,딸바 생과일요플레,2020-12-30 23:40:53.006719


## 연령대, 성별 메뉴 추천

In [14]:
tmpdf = pd.merge(sales_df, cus_df, on='이름')
tmpdf.sample(10)

,이름,메뉴,판매시간,성별,연령대
18147,홍성석,더치커피,2020-02-28 18:57:48.479292,0,40
15166,유주영,석류차,2020-05-14 14:33:04.679577,1,30
59497,오창석,페퍼민트,2019-12-19 01:39:00.658460,0,20
58635,오창석,돌체라떼,2019-06-09 07:49:18.898548,0,20
71536,한영재,매실차,2019-05-12 08:09:14.104658,1,20
5924,서준호,다방커피,2020-08-30 07:28:58.628179,0,20
87544,정석원,그린티라떼,2019-04-22 22:15:25.317812,0,20
73964,한영재,청포도 요거트,2020-10-18 08:45:13.986689,1,20
70251,권재경,카페모카,2020-07-20 13:56:39.407825,1,30
9916,김형건,토마토 생과일요플레,2019-02-07 19:06:42.431252,0,30


In [15]:
tmpdf.groupby('연령대')['메뉴']

In [16]:
tmpdf.groupby(by=['연령대', '성별', '메뉴']).size()

연령대  성별  메뉴        
20   0   고구마라떼         396
         그린티 프라푸치노     398
         그린티라떼         396
         다방커피          416
         더치라떼          367
                      ... 
40   1   토마토 생과일요플레     35
         토마토 생과일주스      33
         페퍼민트           33
         플레인 스무디        51
         플레인 요거트        37
Length: 438, dtype: int64

In [17]:
df = pd.DataFrame(tmpdf.groupby(by=['연령대', '성별', '메뉴']).size()).reset_index()
df

,연령대,성별,메뉴,0
0,20,0,고구마라떼,396
1,20,0,그린티 프라푸치노,398
2,20,0,그린티라떼,396
3,20,0,다방커피,416
4,20,0,더치라떼,367
...,...,...,...,...
433,40,1,토마토 생과일요플레,35
434,40,1,토마토 생과일주스,33
435,40,1,페퍼민트,33
436,40,1,플레인 스무디,51


In [18]:
age = 30
sex = 0

In [19]:
recommand = df[ (df['연령대'] == age) & ( df['성별'] == sex ) ].sort_values(by=0, ascending=False).head()
rec_menu = list(recommand.메뉴)
recommand

,연령대,성별,메뉴,0
163,30,0,로얄밀크티,207
177,30,0,소이라뗴,200
171,30,0,블루레몬 에이드,200
214,30,0,토마토 생과일요플레,199
206,30,0,카라멜모카,193


In [20]:
if sex == 0:
    gender = "남성"
else:
    gender = "여성"
print(f"{age}대 {gender}들이 가장 많이 선택하는 메뉴는")
print("="*30)
for menu in rec_menu:
    print(menu)
print("="*30)
print("입니다.")

30대 남성들이 가장 많이 선택하는 메뉴는
로얄밀크티
소이라뗴
블루레몬 에이드
토마토 생과일요플레
카라멜모카
입니다.


In [21]:
# 전체 연령대 성별 메뉴 추천
ages = list(df.연령대.unique())
sexs = list(df.성별.unique())

for age in ages:
    for sex in sexs:
        recommand = df[ (df['연령대'] == age) & ( df['성별'] == sex ) ].sort_values(by=0, ascending=False).head()
        rec_menu = list(recommand.메뉴)
        if sex == 0:
            gender = "남성"
        else:
            gender = "여성"
        print(f"{age}대 {gender}들이 가장 많이 선택하는 메뉴는")
        print("="*30)
        for menu in rec_menu:
            print(menu)
        print("="*30)
        print("입니다.")
        print("-"*30)

20대 남성들이 가장 많이 선택하는 메뉴는
딸바 생과일주스
민트레몬 에이드
소이라뗴
딸기 생과일요플레
플레인 요거트
입니다.
------------------------------
20대 여성들이 가장 많이 선택하는 메뉴는
카푸치노
에스프레소
유자차
자몽 퓨레에이드
고구마라떼
입니다.
------------------------------
30대 남성들이 가장 많이 선택하는 메뉴는
로얄밀크티
소이라뗴
블루레몬 에이드
토마토 생과일요플레
카라멜모카
입니다.
------------------------------
30대 여성들이 가장 많이 선택하는 메뉴는
딸기 생과일주스
블루베리 퓨레에이드
오렌지 생과일주스
자몽 생과일주스
모카치노
입니다.
------------------------------
40대 남성들이 가장 많이 선택하는 메뉴는
페퍼민트
로얄밀크티
딸바 생과일주스
블루베리 퓨레에이드
바나나 생과일주스
입니다.
------------------------------
40대 여성들이 가장 많이 선택하는 메뉴는
오렌지 생과일주스
얼그레이
캐모마일
청포도 요거트
키위 생과일요플레
입니다.
------------------------------


## 개인별 메뉴 추천

In [22]:
df = pd.DataFrame(sales_df.groupby(['이름','메뉴']).size()).reset_index()
df

,이름,메뉴,0
0,권재경,고구마라떼,39
1,권재경,그린티 프라푸치노,47
2,권재경,그린티라떼,54
3,권재경,다방커피,48
4,권재경,더치라떼,41
...,...,...,...
2258,홍성석,토마토 생과일요플레,53
2259,홍성석,토마토 생과일주스,53
2260,홍성석,페퍼민트,42
2261,홍성석,플레인 스무디,44


In [23]:
name = "권재경"
df[ (df['이름'] == name) ].sort_values(by=0, ascending=False).head(3)

,이름,메뉴,0
8,권재경,딸기 생과일주스,64
28,권재경,블루베리 퓨레에이드,60
57,권재경,초코민트라떼,58


In [24]:
rec_menu = list(df[ (df['이름'] == name) ].sort_values(by=0, ascending=False).head(3).메뉴)

In [25]:
print(f"{name}님이 자주 찾으시는 메뉴는")
print("="*30)
for menu in rec_menu:
    print(menu)
print("="*30)
print("입니다.")

권재경님이 자주 찾으시는 메뉴는
딸기 생과일주스
블루베리 퓨레에이드
초코민트라떼
입니다.


In [26]:
# 개인별 메뉴 추천
names = list(df.이름.unique())

for name in names:
    rec_menu = list(df[ (df['이름'] == name) ].sort_values(by=0, ascending=False).head(3).메뉴)
    print(f"{name}님이 자주 찾으시는 메뉴는")
    print("="*30)
    for menu in rec_menu:
        print(menu)
    print("="*30)
    print("입니다.")
    print("-"*30)

권재경님이 자주 찾으시는 메뉴는
딸기 생과일주스
블루베리 퓨레에이드
초코민트라떼
입니다.
------------------------------
김구님이 자주 찾으시는 메뉴는
키위 생과일요플레
석류차
카페모카
입니다.
------------------------------
김민섭님이 자주 찾으시는 메뉴는
딸바 생과일주스
다방커피
레몬차
입니다.
------------------------------
김봉섭님이 자주 찾으시는 메뉴는
딸바 생과일주스
블루베리 퓨레에이드
카페라뗴
입니다.
------------------------------
김선우님이 자주 찾으시는 메뉴는
딸바 생과일주스
그린티 프라푸치노
자몽 생과일요플레
입니다.
------------------------------
김이환님이 자주 찾으시는 메뉴는
오렌지 퓨레에이드
밀크쉐이크
카페모카
입니다.
------------------------------
김형건님이 자주 찾으시는 메뉴는
토마토 생과일요플레
바나나 생과일주스
카라멜모카
입니다.
------------------------------
김형욱님이 자주 찾으시는 메뉴는
오렌지 생과일주스
블루베리 퓨레에이드
자몽차
입니다.
------------------------------
김효원님이 자주 찾으시는 메뉴는
석류 에이드
자몽 생과일요플레
모카치노
입니다.
------------------------------
류아인님이 자주 찾으시는 메뉴는
카페라뗴
카푸치노
그린티라떼
입니다.
------------------------------
박경민님이 자주 찾으시는 메뉴는
레몬 에이드
오렌지 생과일요플레
청포도 퓨레에이드
입니다.
------------------------------
박성훈님이 자주 찾으시는 메뉴는
플레인 요거트
소이라뗴
딸바 생과일요플레
입니다.
------------------------------
박소현님이 자주 찾으시는 메뉴는
딸기 요거트
카라멜마끼아또
유자차
입니다.
------------------------------
